# Codes

In [1]:
import numpy as np

class Simplex(object):
    def __init__(self, obj, b_matrix=[], maxloop=1000, max_mode=False):
        self.mat, self.max_mode = np.array(
            [[0] + obj]) * (-1 if max_mode else 1), max_mode
        if b_matrix != []:
            self.mat = np.vstack([self.mat, b_matrix])
        self.count = 0
        self.maxloop = maxloop

    def _pivot(self, mat, B, row, col):
        mat[row] /= mat[row][col]
        ids = np.arange(mat.shape[0]) != row
        # for each i!= row do: mat[i]= mat[i] - mat[row] * mat[i][col]
        mat[ids] -= mat[row] * mat[ids, col:col + 1]
        B[row] = col
        self.count += 1
        # print(self.count)

    def _simplex(self, mat, B, m, n):
        while mat[0, 1:].min() < 0 and self.count < self.maxloop:
            # use Bland's method to avoid degeneracy
            # col = np.where(mat[0, 1:] < 0)[0][0] + 1
            # use Dantzig's method, may encounter degeneracy
            delta = []
            for col in (np.where(mat[0, 1:] < 0)[0] + 1):
                row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                                range(1, mat.shape[0])]).argmax() + 1
                delta.append(-mat[0][col]/mat[row][col]*mat[row][0])
                
            col = (np.where(mat[0, 1:] < 0)[0] + 1)[np.array(delta).argmin()]
            row = np.array([mat[i][0] / mat[i][col] if mat[i][col] > 0 else 0x7fffffff for i in 
                range(1, mat.shape[0])]).argmin() + 1
            if mat[row][col] <= 0:
                return None  # the theta is ∞, the problem is unbounded
            self._pivot(mat, B, row, col)
        print('loop:', self.count)
        return mat[0][0] * (1 if self.max_mode else -1), {B[i]: mat[i, 0] for i in range(1, m) if B[i] < n}

    def solve(self):
        m, n = self.mat.shape  # m - 1 is the number slack variables we should add
        temp, B = np.vstack([np.zeros((1, m - 1)), np.eye(m - 1)]
                            ), list(range(n - 1, n + m - 1))  # add diagonal array
        mat = self.mat = np.hstack([self.mat, temp])  # combine them!
        if mat[1:, 0].min() < 0:  # is the initial basic solution feasible?
            row = mat[1:, 0].argmin() + 1  # find the index of min b
            # set first row value to zero, and store the previous value
            temp, mat[0] = np.copy(mat[0]), 0
            mat = np.hstack(
                [mat, np.array([1] + [-1] * (m - 1)).reshape((-1, 1))])
            self._pivot(mat, B, row, mat.shape[1] - 1)
            if self._simplex(mat, B, m, n)[0] != 0:
                return None  # the problem has no answer
            if mat.shape[1] - 1 in B:  # if the x0 in B, we should pivot it.
                self._pivot(mat, B, B.index(
                    mat.shape[1] - 1), np.where(mat[0, 1:] != 0)[0][0] + 1)
            # recover the first line
            self.mat = np.vstack([temp, mat[1:, :-1]])
            for i, x in enumerate(B[1:]):
                self.mat[0] -= self.mat[0, x] * self.mat[i + 1]
        return self._simplex(self.mat, B, m, n)

# Test: AGG


In [2]:
%%time
data = np.loadtxt("data_b.txt", dtype=int)
Z = list(data[0][1:])
B = list(data[1:])
test = Simplex(Z, B)
print(test.solve())
print(test.mat)

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


loop: 95
(-91796798.0, {453: 56200.0, 373: 98200.0, 362: 245714.0, 435: 342923.0, 485: 1201600.0, 421: 2357353.0, 503: 78000.0, 385: 155200.0, 513: 180000.0, 427: 234018.0, 523: 1391199.0, 409: 3567262.0, 538: 450000.0, 431: 891592.0, 359: 56200.0, 393: 143109.0, 361: 0.0, 464: 377143.0, 473: 142824.0, 397: 275066.0, 481: 16000.0, 439: 19101.0, 363: 1201600.0, 441: 2516184.0, 495: 34200.0, 443: 50104.0, 365: 78000.0, 445: 220013.0, 367: 180000.0, 447: 282021.0, 369: 1391199.0, 449: 4291989.0, 371: 450000.0, 451: 841068.0, 454: 42000.0, 433: 331117.0, 375: 377143.0, 465: 114285.0, 377: 142824.0, 437: 469639.0, 379: 16000.0, 482: 4400.0, 381: 1984799.0, 487: 684800.0, 383: 34200.0, 496: 20000.0, 504: 77200.0, 505: 59200.0, 387: 258000.0, 515: 78000.0, 389: 2414999.0, 525: 1220800.0, 391: 706000.0, 540: 260000.0, 455: 134000.0, 456: 134000.0, 395: 491428.0, 466: 1143.0, 474: 145882.0, 475: 197647.0, 399: 20400.0, 483: 5000.0, 401: 2235691.0, 488: 572000.0, 403: 54200.0, 497: 14000.0, 405: